# Matching Code

This code is to implement and test different matching methods

Explain why we did matching? plot average distances of matching. Think of a KPI. Illustrate with one graph or visualisation, this can be a good distance measure, then back up with literature, the ECB article.

Comapre the coverage, do we have more bonds than taking a very strict matching algorithm, Create a table with different matching algorithms and the results. Show that we used different matching methods.

emphasise that the data is scarce, we do not want to lose too much data, get better results with broader scope. 

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/Fijiman001/EGR-Empirical-Project/refs/heads/main/data/Final%20data%20for%20the%20study/df_final_7.csv"
df = pd.read_csv(url)
print(df.columns)
print(df.head())

Index(['Name', 'Clean_Company_Maturity', 'WKN', 'URL', 'ISIN', 'Company',
       'Kupon_Maturity', 'Coupon', 'Maturity_Start', 'Maturity_End',
       'is_green', 'Last Price', 'DateTime', 'Volume in Euro', '+/- %',
       'Currency', 'YTM', 'Modified Duration', 'Kupon', 'Emittent', 'Branche',
       'Fälligkeit', 'Schuldnerkündigungsart', 'Sonderkündigung', 'Nachrangig',
       'Kleinste handelbare Einheit', 'Spezialist', 'Handelsmodell',
       'Emissionsdatum', 'Emissionsvolumen', 'Umlaufendes Volumen',
       'Emissionswährung', 'Depotwährung', 'Notierungsaufnahme',
       'Emission_year_2020.0', 'Emission_year_2021.0', 'Emission_year_2022.0',
       'Emission_year_2023.0', 'Emission_year_2024.0', 'Emission_year_2025.0',
       'Date', 'fälligkeit_datum', 'emission_datum', 'days_to_maturity'],
      dtype='object')
                   Name Clean_Company_Maturity     WKN  \
0  E.ON SE 0,375% 20/27    e on se 0 375 20 27  A254QR   
1  E.ON SE 0,375% 20/27    e on se 0 375 20 27  A254QR

In [2]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances

def match_green_bonds(df):
    # Ensure the dataframe has the necessary columns
    required_cols = ['Company', 'DateTime', 'Coupon', 'days_to_maturity', 'Emission_year_2020.0', 
                    'Emission_year_2021.0', 'Emission_year_2022.0', 
                    'Emission_year_2023.0', 'Emission_year_2024.0', 
                    'Emission_year_2025.0', 'is_green', 'SchuldnerkÃ¼ndigungsart']
    
    # Add is_green column if it doesn't exist (for testing)
    if 'is_green' not in df.columns:
        # This is just for testing - you would replace this with your actual data
        df['is_green'] = np.random.choice([0, 1], size=len(df))
    
    # Create features for matching
    features = ['Coupon', 'days_to_maturity']
    
    # Add emission year as a feature
    emission_cols = [col for col in df.columns if col.startswith('Emission_year_')]
    
    # Create a function to convert emission year columns to a single value
    def get_emission_year(row):
        for col in emission_cols:
            if row[col]:
                # Extract year from column name (e.g., 'Emission_year_2020.0' -> 2020)
                return float(col.split('_')[2].split('.')[0])
        return np.nan
    
    df['emission_year'] = df.apply(get_emission_year, axis=1)
    features.append('emission_year')
    
    # Filter out rows with missing values in crucial columns
    df_clean = df.dropna(subset=features + ['Company', 'is_green'])
    
    # Get unique companies
    companies = df_clean['Company'].unique()
    
    # Prepare results dataframe
    results = []
    
    # For each company, match green bonds with conventional bonds
    for company in companies:
        company_bonds = df_clean[df_clean['Company'] == company]
        
        # Skip if there are no green bonds or no conventional bonds
        if 1 not in company_bonds['is_green'].values or 0 not in company_bonds['is_green'].values:
            continue
        
        green_bonds = company_bonds[company_bonds['is_green'] == 1]
        conv_bonds = company_bonds[company_bonds['is_green'] == 0]
        
        # Extract features for matching
        green_features = green_bonds[features].values
        conv_features = conv_bonds[features].values
        
        # Standardize features
        scaler = StandardScaler()
        green_scaled = scaler.fit_transform(green_features)
        conv_scaled = scaler.transform(conv_features)
        
        # Calculate distances between each green bond and each conventional bond
        distances = euclidean_distances(green_scaled, conv_scaled)
        
        # Match each green bond with the closest conventional bond
        for i, green_idx in enumerate(green_bonds.index):
            closest_conv_idx = conv_bonds.index[np.argmin(distances[i])]
            
            # Create a pair
            green_bond = green_bonds.loc[green_idx]
            conv_bond = conv_bonds.loc[closest_conv_idx]
            
            results.append({
                'Company': company,
                'Green_Bond_Name': green_bond['Name'] if 'Name' in green_bond.index else 'Unknown',
                'Green_Bond_ISIN': green_bond['ISIN'] if 'ISIN' in green_bond.index else 'Unknown',
                'Green_Bond_Coupon': green_bond['Coupon'],
                'Green_Bond_Maturity': green_bond['days_to_maturity'],
                'Green_Bond_Emission_Year': green_bond['emission_year'],
                'Conv_Bond_Name': conv_bond['Name'] if 'Name' in conv_bond.index else 'Unknown',
                'Conv_Bond_ISIN': conv_bond['ISIN'] if 'ISIN' in conv_bond.index else 'Unknown',
                'Conv_Bond_Coupon': conv_bond['Coupon'],
                'Conv_Bond_Maturity': conv_bond['days_to_maturity'],
                'Conv_Bond_Emission_Year': conv_bond['emission_year'],
                'Distance': distances[i][np.argmin(distances[i])]
            })
    
    # Create results dataframe
    results_df = pd.DataFrame(results)
    return results_df

matched_bonds = match_green_bonds(df)

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
matched_bonds

for any given green bond, take the match which has the smallest distance
take average distance. group by bonds and conventional bond unique name, compute mean average distance.
then take smallest pair in average.

weight the coupon in the distance measure a bit less, put more weight on the days to maturity.
include emission date measure. 

Show this in our analysis.

test YTM vs. Z-spread.

In [ ]:
# print(matched_bonds['Green_Bond_ISIN'].nunique()) 100 unique green bond ISINs
# dropping duplicates
matched_bonds.drop_duplicates(inplace=True)
# no longer need other columns other than the ISINS
matched_bonds_ISIN = matched_bonds[['Green_Bond_ISIN', 'Conv_Bond_ISIN', 'Distance']]
matched_bonds_ISIN

# Using this matching method, we get 100 matches

In [ ]:
import pandas as pd
import numpy as np

def prepare_regression_data(matched_bonds, original_df):
    """
    Prepare the regression dataset from matched green and conventional bonds.
    
    Parameters:
    matched_bonds (DataFrame): The matched pairs of green and conventional bonds
    original_df (DataFrame): The original dataset with YTM information
    
    Returns:
    DataFrame: Clean dataset ready for regression analysis
    """
    # Create regression dataset from matched pairs
    regression_data = []
    
    for _, row in matched_bonds.iterrows():
        # Green bond data
        green_data = {
            'ISIN': row['Green_Bond_ISIN'],
            'Company': row['Company'],
            'Coupon': row['Green_Bond_Coupon'],
            'Maturity': row['Green_Bond_Maturity'],
            'Emission_Year': row['Green_Bond_Emission_Year'],
            'is_green': 1,
            'Distance': row['Distance'],
            'Bond_Pair_ID': _  # Use row index as pair identifier
        }
        
        # Conventional bond data
        conv_data = {
            'ISIN': row['Conv_Bond_ISIN'],
            'Company': row['Company'],
            'Coupon': row['Conv_Bond_Coupon'],
            'Maturity': row['Conv_Bond_Maturity'],
            'Emission_Year': row['Conv_Bond_Emission_Year'],
            'is_green': 0,
            'Distance': row['Distance'],
            'Bond_Pair_ID': _  # Same pair identifier
        }
        
        regression_data.append(green_data)
        regression_data.append(conv_data)
    
    # Create dataframe
    reg_df = pd.DataFrame(regression_data)
    
    # Merge with original data to get YTM and DateTime
    reg_df = pd.merge(
        reg_df,
        original_df[['ISIN', 'YTM', 'DateTime']],
        on='ISIN',
        how='left'
    )
    
    # Convert all numeric columns to float
    numeric_cols = ['is_green', 'Coupon', 'Maturity', 'Distance', 'YTM', 'Emission_Year']
    for col in numeric_cols:
        reg_df[col] = pd.to_numeric(reg_df[col], errors='coerce')
    
    # Drop rows with missing values in key columns
    reg_df = reg_df.dropna(subset=['YTM', 'is_green', 'Coupon', 'Maturity', 'Distance'])
    
    # Handle company dummies properly
    # First ensure Company column has proper encoding and clean values
    reg_df['Company'] = reg_df['Company'].astype(str).str.strip()
    
    # Let's print the unique companies to verify
    print(f"Number of unique companies: {reg_df['Company'].nunique()}")
    print(f"Company values: {reg_df['Company'].unique()[:5]}...")  # Show first 5
    
    # Create company dummy variables - ensure they're properly encoded
    company_cols = pd.get_dummies(reg_df['Company'], prefix='Company', drop_first=True)
    
    # Instead of using get_dummies on the entire dataframe, combine manually
    reg_df = pd.concat([reg_df, company_cols], axis=1)
    
    return reg_df

df_regression = prepare_regression_data(matched_bonds, df)

In [ ]:
df_regression

# test regression

In [ ]:
# First prepare data
reg_df = prepare_regression_data(matched_bonds, df)

# For regression analysis, select appropriate columns
X_cols = ['is_green', 'Coupon', 'Maturity', 'Distance']
company_cols = [col for col in reg_df.columns if col.startswith('Company_')]

# Combine main variables with company dummies
X = reg_df[X_cols + company_cols]
y = reg_df['YTM']

# Add constant and run regression
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()